In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow_addons as tfa
from transformers import BertConfig, BertModel, BertTokenizer
import transformers
import tokenizers
import re

In [ ]:
def deconstruct(dtf, type="train"):
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("\n", " "))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("'s", ""))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("'", " "))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("-", " "))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("‘", " "))

    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("”", ""))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: re.sub(r"[0-9]+\–[0-9]+", "VREMYAPROMdsfsf1111",x))
    dtf["excerpt"] = dtf["excerpt"].apply(lambda x: x.replace("°", " temperatura "))
#     TRAIN_TABLE["excerpt"] = TRAIN_TABLE["excerpt"].apply(lambda x: x.replace("—", " "))
    dtf["count_w"] = dtf["excerpt"].apply(lambda x: len(x.split(" ")))
    if type=="train":
        dtf["split"] = dtf["target"].apply(lambda x: math.trunc(x))
    return dtf, dtf["count_w"].max()

In [ ]:
TEST_TABLE = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
TEST_TABLE, _ = deconstruct(TEST_TABLE,"test")
list_test = TEST_TABLE["excerpt"].to_list()
max_lens = 256
max_word = 30000


In [ ]:
tokenizer = BertTokenizer.from_pretrained("../input/tfbert-large-uncased/vocab.txt", do_lower_case=True)
def ret_batch(list_data):
    encoded =  tokenizer.batch_encode_plus(
            list_data,
            add_special_tokens=True,
            max_length=max_lens,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )
    return encoded

In [ ]:
encoded_test = ret_batch(list_test)
datas_test = [ np.array(encoded_test["input_ids"], dtype="int32"),
               np.array(encoded_test["attention_mask"], dtype="int32"),
               np.array(encoded_test["token_type_ids"], dtype="int32")
             ]

In [ ]:
configuration = BertConfig.from_pretrained(f'/kaggle/input/bert-tensorflow/bert-large-uncased-whole-word-masking-finetuned-squad-config.json')
base_path = '/kaggle/input/bert-tensorflow/bert-large-uncased-whole-word-masking-tf_model.h5'

In [ ]:
def get_model():
    input_ids = tf.keras.layers.Input(
        shape=(max_lens,), dtype=tf.int32, name="input_ids"
    )
    attention_masks = tf.keras.layers.Input(
        shape=(max_lens,), dtype=tf.int32, name="attention_masks"
    )
    token_type_ids = tf.keras.layers.Input(
        shape=(max_lens,), dtype=tf.int32, name="token_type_ids"
    )
    bert_model = transformers.TFBertModel.from_pretrained(base_path, config=configuration)
    bert_model.trainable = False
    outputs = bert_model(
        input_ids, attention_masks, token_type_ids
    )
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(256, return_sequences=True, )
    )(outputs[0])
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=True)
    )(bi_lstm)
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.1)(concat)
    output = tf.keras.layers.Dense(1)(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.MeanAbsoluteError(),
        metrics=tf.keras.metrics.RootMeanSquaredError(),
    )
    return model



In [ ]:
model = get_model()
model.load_weights("../input/zefir-book-v1/w2.h5")

In [ ]:
pred = model.predict(datas_test)
TEST_TABLE["target"] = pred
name = "submission.csv"
sub = TEST_TABLE[["id","target"]]
sub.to_csv(name, index=False)